# Planning Area and Subzone Extractor

In [10]:
import numpy as np
import pandas as pd
import json

In [11]:
file = None
with open("formatted-data.json", "r") as f:
    file = f.read()

subzone_data = json.loads(file)

In [12]:
locations_df = pd.read_csv("land_use_data.csv")

In [13]:
from shapely.geometry import Point, Polygon

def is_point_inside_area(current_coordinates, area):
    lat, lon = current_coordinates
    point = Point(lat, lon)
    polygon_coord = area['coordinates']
    if isinstance(polygon_coord[0][0], list):
        polygon_coord = polygon_coord[0]
        polygon_coord = [[lat, lon] for lat, lon, _ in polygon_coord]
    if len(polygon_coord) < 4:
        return False
    polygon = Polygon(polygon_coord)
    return polygon.contains(point)

In [14]:
from tqdm import tqdm

appended_points = []

subzones = []
planning_areas = []
regions = []

for i in tqdm(range(len(locations_df)), desc="Processing Subzone Area Data"):

    latitude = locations_df.loc[i, 'latitude']
    longitude = locations_df.loc[i, 'longitude']

    updated = False

    for subzone in subzone_data:
        if is_point_inside_area([latitude, longitude], subzone):
            subzones.append(subzone['subzone'])
            planning_areas.append(subzone['planning_area'])
            regions.append(subzone['region'])
            updated = True
            break
    
    if updated: continue
    
    subzones.append("UNKNOWN")
    planning_areas.append("UNKNOWN")
    regions.append("UNKNOWN")

locations_df['subzone'] = subzones
locations_df['planning_area'] = planning_areas
locations_df['region'] = regions

Processing Subzone Area Data: 100%|██████████| 1406/1406 [00:26<00:00, 53.65it/s]


In [15]:
locations_df

,latitude,longitude,land_use_label,distance_to_waterbody,distance_to_open_space,subzone,planning_area,region
0,1.341371,103.816937,OPEN SPACE,0.001572,0.000000,CENTRAL WATER CATCHMENT,CENTRAL WATER CATCHMENT,NORTH REGION
1,1.341741,103.834931,ROAD,0.001046,0.000068,CENTRAL WATER CATCHMENT,CENTRAL WATER CATCHMENT,NORTH REGION
2,1.376230,103.734428,PLACE OF WORSHIP,0.000681,0.000333,FOREST HILL,TENGAH,WEST REGION
3,1.404280,103.866961,ROAD,0.009188,0.001341,SELETAR AEROSPACE PARK,SELETAR,NORTH-EAST REGION
4,1.330695,103.639991,BUSINESS 2,0.002164,0.015768,TUAS NORTH,TUAS,WEST REGION
...,...,...,...,...,...,...,...,...
1401,1.372259,103.917054,ROAD,0.004993,0.005235,PLAB,PAYA LEBAR,EAST REGION
1402,1.331814,103.630235,WATERBODY,0.000000,0.018562,TUAS PROMENADE,TUAS,WEST REGION
1403,1.346468,104.028241,RESERVE SITE,0.009619,0.000282,UNKNOWN,UNKNOWN,UNKNOWN
1404,1.282489,103.780019,UTILITY,0.006392,0.000158,PORT,QUEENSTOWN,CENTRAL REGION


In [16]:
locations_df.to_csv("land_use_data.csv", index=False)